<a href="https://colab.research.google.com/github/PremKarhale/Full-stack-using-Agentic-AI-/blob/main/AI_powered_fitness_planner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **AI POWERED FITNESS APP PLANNER**

In [3]:
!pip install -qU langchain_google_genai langchain langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.8/105.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 713.3/713.3 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.1/489.1 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.9/234.9 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google

# **Tool** - for calculating metrics (BMI,BMR,required calaroies)

In [9]:
weight=int(input('Enter your weight in kg: '))
height_m=float(input('Enter your height in meters: '))
age=int(input('Enter your age:'))
gender=input('Enter your gender: ')
activity_level=input('Enter your activity level form sedentary/light/moderate/active : ')
weight_goal=input('Enter your weight goal from lose weight/gain weight/Maintain: ')

Enter your weight in kg: 60
Enter your height in meters: 1.7
Enter your age:21
Enter your gender: male
Enter your activity level form sedentary/light/moderate/active : active
Enter your weight goal from lose weight/gain weight/Maintain: gain weight


In [10]:
# tool for calculating BMI or BMR
from ast import Call
from langchain_core.tools import tool

@tool
def BMI_Calculator(
    weight: float,
    height_m: float,
    age: int,
    gender: str,
    activity_level: str,
    weight_goal: str
) -> dict:
    """
    Calculates BMI, category, BMR, and maintenance calories
    """

    # ---------- BMI ----------
    bmi = weight / (height_m ** 2)

    if bmi < 18.5:
        bmi_category = "Underweight"
    elif bmi < 25:
        bmi_category = "Normal"
    elif bmi < 30:
        bmi_category = "Overweight"
    else:
        bmi_category = "Obesity"

    # ---------- BMR ----------
    height_cm = height_m * 100

    if gender.lower() == 'male':
        bmr = (10 * weight) + (6.25 * height_cm) - (5 * age) + 5
    else:
        bmr = (10 * weight) + (6.25 * height_cm) - (5 * age) - 161

    # ---------- Activity Multiplier ----------
    activity_factors = {
        'sedentary': 1.2,
        'light': 1.375,
        'moderate': 1.55,
        'active': 1.725
    }

    factor = activity_factors.get(activity_level.lower(), 1.2)
    maintenance_calories = bmr * factor

    # ---------- Weight Goal Adjustment ----------
    if weight_goal.lower() == 'lose weight':
        maintenance_calories -= 500
    elif weight_goal.lower() == 'gain weight':
        maintenance_calories += 500

    # ---------- Final Output ----------
    return {
        "BMI": round(bmi, 2),
        "BMI_Category": bmi_category,
        "BMR": round(bmr, 2),
        "Daily_calories as per goal": round(maintenance_calories, 2)
    }



In [1]:
from google.colab import userdata
gemini_api=userdata.get('GEMINI_API_KEY')

# Tool Binding

In [13]:
# llm testing
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
llm=ChatGoogleGenerativeAI(model='gemini-2.5-flash',google_api_key=gemini_api)

prompt=PromptTemplate(template='this are user details {weight},{height_m},{age},{gender},{activity_level},{weight_goal} based on this usr data give me the desired results',input_variables=[weight,height_m,age,gender,activity_level,weight_goal])

llm_with_tool = llm.bind(tools=[BMI_Calculator])

chain = prompt | llm_with_tool




- llm tells us with tool to use with kargs , we have to invoke the desired tool by overself

In [18]:
response=chain.invoke({'weight':weight,'height_m':height_m,'age':age,'gender':gender,'activity_level':activity_level,'weight_goal':weight_goal})

 # Tool Execution

In [19]:
print(response.tool_calls[0])

{'name': 'BMI_Calculator', 'args': {'weight': 60, 'height_m': 1.7, 'activity_level': 'active', 'gender': 'male', 'weight_goal': 'gain weight', 'age': 21}, 'id': '86a94667-d390-424c-9f42-78c1716cb5bc', 'type': 'tool_call'}


In [23]:
metrics = BMI_Calculator.invoke(response.tool_calls[0])
print(metrics.content)

{"BMI": 20.76, "BMI_Category": "Normal", "BMR": 1562.5, "Daily_calories as per goal": 3195.31}


generate the 7 day workout and diet plan / chains or RunnableParallel